<h1 style='color: green; font-size: 30px; font-weight: bold;'>Data Science - Regressão Linear: de baixo dos panos</h1>

# <font color='black' style='font-size: 24px;'>1.1 Conhecendo o Dataset</font>
<hr style='border: 2px solid black;'>

## Importando o numpy, pandas e random

In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

## O Dataset e o Projeto
<hr>

### Descrição:
<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>O mercado imobiliário vem sendo objeto de diversos estudos e pesquisas nos últimos tempos. A crise financeira que afeta a economia tem afetado significativamente os investimentos e ganhos advindos deste setor. Este cenário incentiva o aumento do interesse por estudos de previsão de demanda baseados em características deste mercado, dos imóveis e do entorno destes imóveis.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Neste contexto o objetivo principal do nosso projeto é desenvolver um sistema de avaliação imobiliária utilizando a metodologia de regressões lineares que é uma das técnicas de machine learning.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Nosso dataset é uma amostra aleatória de tamanho 5000 de imóveis disponíveis para venda no município do Rio de Janeiro.</p>

### Dados:
<ul style='font-size: 15px; line-height: 2; text-align: justify;'>
    <li><b>Valor</b> - Valor (R$) de oferta do imóvel;</li>
    <li><b>Area</b> - Área do imóvel em m².</li>
    <li><b>Dist_Praia</b> - Distância do imóvel até a praia (km) (em linha reta).</li>
    <li><b>Dist_Farmacia</b> - Distância do imóvel até a farmácia mais próxima (km) (em linha reta).</li>
</ul>

In [2]:
dados = pd.read_csv('dados_regressao_multivariavel.csv')

In [3]:
dados.head()

,Valor,Area,Dist_Praia,Dist_Farmacia
0,4600000,280,0.240925,0.793637
1,900000,208,0.904136,0.134494
2,2550000,170,0.059525,0.423318
3,550000,100,2.883181,0.525064
4,2200000,164,0.239758,0.192374


In [4]:
valor = np.log(dados['Valor'])
area = np.log(dados['Area'])
dist_Praia = np.log(dados['Dist_Praia'] + 1)
dist_Farmacia = np.log(dados['Dist_Farmacia'] + 1)

valor = np.array(valor)
area = np.array(area)
dist_Praia = np.array(dist_Praia)
dist_Farmacia = np.array(dist_Farmacia)

X = np.array([area, dist_Praia, dist_Farmacia]).T
y = valor

# Regressão multivariável

Desta vez, não temos apenas uma variável. Então, nosso problema vai ficar um pouco maior, em vez de otimizarmos dois parâmetros, devemos otimizar $n+1$ parâmetros, sendo $n$ o número de variáveis do problema. No nosso caso, temos três variáveis no problema, portanto iremos otimizar no total quatro parâmetros. Então nossa hipótese de forma geral será:

$$
y(x_0, x_1, ..., x_n) = \theta_0 + \theta_1 x_0 + \theta_2 x_1 + ... + \theta_{n-1} x_n
$$

Como nosso problema possuí três variáveis, teremos:

$$
y(x_0, x_1, x_2) = \theta_0 + \theta_1 x_0 + \theta_2 x_1 + \theta_3 x_2 + \theta_4 x_3
$$

Podemos definir isto a partir da notação matricial:

$$
y(X) = \theta_0 + X \bullet \theta 
$$

Desta forma, é facil prever usando nosso modelo.

In [5]:
def prever(theta, X):
    return np.dot(X, theta)

Agora, vamos a função de custo. Usaremos o erro quadrático médio.

$$
EQM = \frac{1}{N}\sum_{i=0}^{N}(y_{i}-y_{i}^{p}(x_{i}))^2
$$

In [6]:
def erro_quadratico_medio(theta, X, y):
    return 1/len(X) * sum([(y_i - prever(theta, x_i)) ** 2 for x_i, y_i in zip(X, y)])

Devemos minimizar nossa função de custo, para isto usamos o gradiente descendente. Nele, usamos a derivada de nossa função de custo, definida como:

$$
EQM' = \frac{1}{2N}\sum_{i=0}^{N}(y^{p}_{i}(x_i) - y_{i})
$$

In [7]:
def gradienteDescendente(theta, X, y, alpha):
    previsto = prever(theta, X)
    erro = previsto - y
    
    gradiente = np.dot(X.T, erro) / (2*len(X))
    
    theta -= alpha*gradiente

O métoto c_ do numpy vai apenas colocar mais um atributo em todas amostras de nosso conjunto. Este ultimo atributo será sempre unitário, e será responsável pela variável independente $\theta_0$.

Após isto, vamos criar nossos pesos de forma aleatória e executar a nossa otimização.

In [15]:
X = np.c_[np.ones(X.shape[0]), X] 

theta = np.random.rand(X.shape[1])

for i in range(10000):
    previsto = prever(theta, X)
    custo = erro_quadratico_medio(theta, X, y)
    
    if i % 1000 == 0:
        print(custo)
    
    gradienteDescendente(theta, X, y, 0.05)

144.63549868740094
0.4300588989408325
0.261538276904437
0.19900141414163008
0.1755473591459858
0.16672243097804956
0.1633986401836649
0.16214640362716784
0.16167458125397785
0.16149680136230554


In [9]:
theta

array([ 4.2474119 ,  2.00964183, -0.15927647,  0.70901717])

In [18]:
def r_2(theta, X, y):
    previsto = prever(theta, X)
    
    variancia_prevista = sum([(y[i] - previsto[i]) ** 2 for i in range(len(X))])
    variancia_original = sum([(y[i] - y.mean()) ** 2 for i in range(len(X))])
    
    return 1 - variancia_prevista / variancia_original

In [19]:
r_2(theta, X, y)

0.8022551474231682

# Comparando com o Sk-learn

In [12]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [22]:
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [24]:
from sklearn.metrics import r2_score

r2_score(y, lr.predict(X))

0.8023047600430262